In [9]:
import os

import pandas as pd
import plotly.express as px
# Install & Import Dependencies
import wandb

pd.options.plotting.backend = "plotly"
api = wandb.Api()

In [3]:
# Define WandB Parameters
entity = "phys_inversion"
# Fetch all projects for the authenticated user
projects = api.projects(entity=entity)

projects = [project for project in projects if "eval_results_" in project.name]

rename_metrics = {
    "env/Env reach_success": "reach_success",
    "env/Env reach_distance": "reach_distance",
    "core/Episode Length": "episode_length",
    "core/Episode Reward": "episode_reward"
}

In [4]:
# Fetch Data from WandB

def get_runs(entity, project, filters, keys=None):
    runs = api.runs(f"{entity}/{project}", filters=filters)
    data = []
    for run in runs:
        history = run.history(keys=keys, pandas=True)  # Adjust as needed

        # Fetch hyperparameters
        config = run.config
        history = pd.concat([history, pd.json_normalize(config, sep='.')], axis=1)

        history["run_name"] = run.name
        history["run_id"] = run.id

        data.append(history)
    return pd.concat(data).reset_index() if data else pd.DataFrame()


In [5]:
filters = {"state": {"$eq": "finished"}}
# Fetch all runs for the project
project_dfs = [get_runs(entity, project.name, filters) for project in projects]

In [13]:
for df in project_dfs:
    df.rename(columns=rename_metrics, inplace=True)
    # multiply by 100 to get percentage
    # df['reach_success'] = df['reach_success'] * 100
    # check if algo type is null, if so then if env.config.disable_discriminator is True then it's PPO else it's AMP
    df['algo_type'] = df.apply(lambda x:
                           x['algo_type'] if pd.notna(x['algo_type'])
                           else ('PPO' if x['env.config.disable_discriminator'] else 'AMP'), axis=1)

In [31]:
for df, project in zip(project_dfs, projects):
    reach_success = df.groupby(["algo_type", "prior"], dropna=False).agg({"reach_success": ["mean", "std"]})
    os.makedirs("eval_results", exist_ok=True)
    reach_success.to_csv(f"eval_results/{project.name}_reach_success.csv", float_format="%.4f")
